# 월간 데이콘 법원 판결 예측 AI 경진대회
Dataset Info.
* ID : 사건 샘플 ID
* first_party : 사건의 첫 번째 당사자
* second_party : 사건의 두 번째 당사자
* facts : 사건 내용
* first_party_winner : 첫 번째 당사자의 승소 여부 (0 : 패배, 1 : 승리)

**첫 번째 당사자의 승소 여부 예측하기**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **1. Importing libraries**

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC


In [4]:
import nltk # 문장 토크나이저

# 영어 불용어 - 불용어 모아 놓은 리스트 다운로드해 제거
nltk.download('all')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize # 토큰화
from nltk.stem.porter import PorterStemmer # 어근 동일화 <-> 이거 말고도 "Lancaster Stemmer"

# 표제어 추출
from nltk.stem import WordNetLemmatizer

# 정규표현 처리
import re

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [5]:
# 데이터 늘린 걸로 한번해보자,,,ㅎㅎ
train_issuedarea = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/train1_issuedarea.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/train1_issuedarea.csv')
train = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/train.csv')
# test = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/test.csv')

In [6]:
train_issuedarea.head(2)

,Unnamed: 0,ID,first_party,second_party,facts1,issued_area,first_party_winner,first_party_ner,second_party_ner
0,0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, p1 a candidate for public of...",others,1,PERSON,PERSON
1,1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,criminal,0,PERSON,PERSON


In [7]:
train.head(2)

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0


In [8]:
train['issued_area'] = train_issuedarea['issued_area']
train['issued_area'] = train_issuedarea['issued_area']
train['first_party_ner'] = train_issuedarea['first_party_ner']
train['second_party_ner'] = train_issuedarea['second_party_ner']

In [9]:
train = train.replace('NORP', 'other')
train = train.replace('CARDINAL', 'other')
train = train.replace('LOC', 'other')
train = train.replace('PRODUCT', 'other')
train = train.replace('LAW', 'other')
train = train.replace('MONEY', 'other')
train = train.replace('WORK_OF_ART', 'other')
train = train.replace('LANGUAGE', 'other')
train = train.replace('DATE', 'other')
train = train.replace('FAC', 'other')

In [10]:
train = train.fillna('other')

In [11]:
train['issued_area'].value_counts()

others      1816
criminal     485
civil        177
Name: issued_area, dtype: int64

train, test split

In [12]:
import random

In [13]:
from sklearn.model_selection import train_test_split


In [14]:
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)


In [15]:
train_df.head(2)

,ID,first_party,second_party,facts,first_party_winner,issued_area,first_party_ner,second_party_ner
2464,TRAIN_2464,United States,Henry Ogle Watson,"On August 17, 1972, a postal inspector receive...",1,others,GPE,PERSON
1561,TRAIN_1561,Vermont,Michael Brillon,"In June 2004 after 3 years awaiting trial, Mic...",1,others,GPE,PERSON


In [16]:
val_df.head(2)

,ID,first_party,second_party,facts,first_party_winner,issued_area,first_party_ner,second_party_ner
1753,TRAIN_1753,Gerald E. Mansell,Gaye M. Mansell,Major Gerald E. Mansell and Gaye M. Mansell w...,1,civil,PERSON,PERSON
259,TRAIN_0259,Freedman,Maryland,Maryland required that all films be submitted ...,1,others,other,GPE


# **2. 파생변수 생성**

* first_party_win_percent	: 첫 번째 당사자가 승소할 확률. 출현 빈도가 2이상일때만 반영함. 1일때는 0.5
* test에서는 first_party가 train의 first_party와 겹치는 경우에는 해당 사람의 승소할 확률을 반영함. 겹치지 않는 경우는 0.5로 반영함.

In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [18]:
len(train_df)

1982

In [19]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [20]:
# winner: 해당 사건에서 승소한 사람
# frequency: winner의 총 등장 횟수
# win_percentage: 승소한 사람이 다른 사건에서도 승소할 확률

train_df['winner']=0
train_df['winner_frequency']=0
train_df['win_percentage']=0

for i in range(len(train_df)):
  if train_df['first_party_winner'][i]==1:
    train_df['winner'][i] = train_df['first_party'][i]
  else:
    train_df['winner'][i] = train_df['second_party'][i]

#for i in range(len(train)):
#  train['winner_frequency'][i] = len(train.loc[train['first_party'] == train['winner'][i]]) + len(train.loc[train['second_party'] == train['winner'][i]])
#  train['win_percentage'][i]= len(train.loc[train['winner'] == train['winner'][i]]) /(len(train.loc[train['first_party'] == train['winner'][i]]) + len(train.loc[train['second_party'] == train['winner'][i]]))

In [21]:
# first_party_frequency: first_party가 나타난 빈도
# first_party_win_percent: first_party가 승소할 확률 / 출현 빈도가 2이상일때만 반영함. 1일때는 0.5
train_df['first_party_win_percent'] = 0
train_df['first_party_frequency'] = 0

for i in range(len(train_df)):
  train_df['first_party_frequency'][i] = len(train_df.loc[train_df['first_party'] == train_df['first_party'][i]]) + len(train_df.loc[train_df['second_party'] == train_df['first_party'][i]])

  if (len(train_df.loc[train_df['first_party'] == train_df['first_party'][i]]) + len(train_df.loc[train_df['second_party'] == train_df['first_party'][i]])) > 1:
    train_df['first_party_win_percent'][i]= len(train_df.loc[train_df['winner'] == train_df['first_party'][i]]) /(len(train_df.loc[train_df['first_party'] == train_df['first_party'][i]]) + len(train_df.loc[train_df['second_party'] == train_df['first_party'][i]]))
  else:
    train_df['first_party_win_percent'][i] = 0.5

In [22]:
train_df.tail(2)

,ID,first_party,second_party,facts,first_party_winner,issued_area,first_party_ner,second_party_ner,winner,winner_frequency,win_percentage,first_party_win_percent,first_party_frequency
1980,TRAIN_1294,Americold Realty Trust,"Conagra Foods, Inc., et al.","The plaintiffs, food distributors including Co...",0,others,ORG,ORG,"Conagra Foods, Inc., et al.",0,0,0.5,1
1981,TRAIN_0860,Bay Area Laundry & Dry Cleaning Pension Trust ...,"Ferbar Corporation of California, Inc.",The Multiemployer Pension Plan Amendments Act ...,1,others,ORG,ORG,Bay Area Laundry & Dry Cleaning Pension Trust ...,0,0,0.5,1


In [23]:
#train=train.drop(['winner',	'winner_frequency',	'win_percentage',	'loser',	'lose_frequency',	'lose_percentage'],axis=1)
train_df=train_df.drop(['winner',	'winner_frequency',	'win_percentage'],axis=1)

In [24]:
train_df

,ID,first_party,second_party,facts,first_party_winner,issued_area,first_party_ner,second_party_ner,first_party_win_percent,first_party_frequency
0,TRAIN_2464,United States,Henry Ogle Watson,"On August 17, 1972, a postal inspector receive...",1,others,GPE,PERSON,0.567742,310
1,TRAIN_1561,Vermont,Michael Brillon,"In June 2004 after 3 years awaiting trial, Mic...",1,others,GPE,PERSON,0.500000,1
2,TRAIN_2469,Ferguson,City of Charleston,After an increase in the use of cocaine by pat...,1,criminal,ORG,GPE,0.500000,1
3,TRAIN_1513,"Anthony R. Tanner, William M. Conover",United States,Anthony Tanner and William Conover were indict...,1,criminal,PERSON,GPE,0.500000,1
4,TRAIN_1133,"James Dawson, et ux.","Dale W. Steager, West Virginia State Tax Commi...",West Virginia Code 11-21-12(c)(6) (“Section 12...,1,others,PERSON,GPE,0.500000,1
...,...,...,...,...,...,...,...,...,...,...
1977,TRAIN_1638,BOE of Westside Community Schools,Mergens By and Through Mergens,The school administration at Westside High Sch...,0,others,ORG,PERSON,0.500000,1
1978,TRAIN_1095,"Sallie M. Hadnott, et al.","Mabel S. Amos, et al.",The National Democratic Party of Alabama (NDPA...,1,others,PERSON,PERSON,0.500000,1
1979,TRAIN_1130,Brosseau,Haugen,"Brosseau, a police officer in Washington state...",1,others,ORG,other,0.500000,1
1980,TRAIN_1294,Americold Realty Trust,"Conagra Foods, Inc., et al.","The plaintiffs, food distributors including Co...",0,others,ORG,ORG,0.500000,1


In [26]:
val_df_first_party = val_df['first_party']

In [27]:
train_df_first_party = train_df[['first_party','first_party_win_percent']]
train_df_first_party=train_df_first_party.drop_duplicates()

In [28]:
test_first_party_병합 = pd.merge(val_df_first_party, train_df_first_party, how='left')

In [29]:
test_first_party_병합 = test_first_party_병합.fillna(0.5)

In [30]:
val_df['first_party_win_percent'] = test_first_party_병합['first_party_win_percent']

In [31]:
def categorize_probability(x):
    if x == 0:
        return "0"
    elif 0.12 <= x <= 0.27:
        return "1"
    elif 0.27 < x < 0.5:
        return "2"
    elif x == 0.5:
        return "3"
    elif 0.5 < x <= 0.55:
        return "4"
    elif 0.55 < x <= 0.66:
        return "5"
    elif 0.66 < x <= 0.83:
        return "6"
    elif x == 1:
        return "7"
    else:
        return "-1"

In [33]:
train_df['승률범주형분류'] = ""

In [34]:
for i in range(len(train_df)):
  train_df['승률범주형분류'][i] = categorize_probability(train_df['first_party_win_percent'][i])

In [35]:
train_df['승률범주형분류'].value_counts()

3    1640
5     138
7      70
6      59
2      41
0      16
1      15
4       3
Name: 승률범주형분류, dtype: int64

In [37]:
val_df['승률범주형분류'] = ""

In [38]:
for i in range(len(val_df)):
  val_df['승률범주형분류'][i] = categorize_probability(val_df['first_party_win_percent'][i])

In [39]:
val_df['승률범주형분류'].value_counts()

3    433
5     32
6      8
2      6
7      5
0      5
4      4
1      3
Name: 승률범주형분류, dtype: int64

In [44]:
train_df=train_df.drop(['first_party_frequency'],axis=1)

In [45]:
train_df.columns

Index(['ID', 'first_party', 'second_party', 'facts', 'first_party_winner',
       'issued_area', 'first_party_ner', 'second_party_ner',
       'first_party_win_percent', '승률범주형분류'],
      dtype='object')

In [46]:
val_df.columns

Index(['ID', 'first_party', 'second_party', 'facts', 'first_party_winner',
       'issued_area', 'first_party_ner', 'second_party_ner',
       'first_party_win_percent', '승률범주형분류'],
      dtype='object')

In [47]:
lst_stopwords = nltk.corpus.stopwords.words("english")
ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()

def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in
                    lst_stopwords]

    ## 어간추출 Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## 표제어 추출 Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## back to string from list
    text = " ".join(lst_text)
    return text

In [48]:
vectorizer = TfidfVectorizer()

def get_vector(vectorizer, df, train_mode):
    # 데이터 클리닝
    df["fact_processing"] = df["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))

    # TF-IDF -- 파라미터 조정 필요?
    if train_mode:
        X_facts = vectorizer.fit_transform(df['fact_processing'])
    else:
        X_facts = vectorizer.transform(df['fact_processing'])

    X_party1 = vectorizer.transform(df['first_party'])
    X_party2 = vectorizer.transform(df['second_party'])

    X_parsed_winlose= vectorizer.transform(df['승률범주형분류'])

    X = np.concatenate([X_party1.todense(), X_party2.todense(), X_facts.todense(), X_parsed_winlose.todense()], axis=1)
    return X

In [49]:
# 벡터화 함수 적용
X = get_vector(vectorizer, train_df, True)
Y = train_df["first_party_winner"]
X_train = np.asarray(X) # np.array 활용 및 적용
Y_train = np.asarray(Y)

In [50]:
model = LogisticRegression()

model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [51]:
X_val = get_vector(vectorizer, val_df, False)
Y_val = val_df["first_party_winner"]
X_val = np.asarray(X_val) # np.array 활용 및 적용
Y_val = np.asarray(Y_val)

In [52]:
model.score(X_val,Y_val)

0.6391129032258065

In [58]:
pred = model.predict(X_val)

In [60]:
pd.DataFrame(pred).value_counts()

1    473
0     23
dtype: int64

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

랜덤포레스트

In [55]:
#logistic
model = RandomForestClassifier()
model.fit(X_train, Y_train)

RandomForestClassifier()

In [56]:
model.score(X_val,Y_val)

0.6310483870967742

KNeighborsClassifier(n_neighbors=3)

In [62]:
model = knn=KNeighborsClassifier(n_neighbors=7)
model.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=7)

In [63]:
model.score(X_val,Y_val)

0.6391129032258065

CATBOOST

In [ ]:
!pip install catboost

In [65]:
from catboost import CatBoostClassifier, Pool

In [66]:
Y.value_counts()

1    1321
0     661
Name: first_party_winner, dtype: int64

In [67]:
counts = list(Y.value_counts())
class_weight = [counts[1]/sum(counts), counts[0]/sum(counts)]
print("weight :", class_weight)

weight : [0.33350151362260344, 0.6664984863773966]


In [68]:
model = CatBoostClassifier(random_seed=42,class_weights=class_weight, verbose=0)
model.fit(X_train, Y_train)

In [69]:
model.score(X_val,Y_val)

0.6431451612903226

#**party 종류 넣을까말까 정하기**